<a href="https://colab.research.google.com/github/debashisdotchatterjee/Circular-Reliability-Paper-1/blob/main/circular_Reliability_Simulation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.stats import vonmises
from scipy.special import i0
import math

# Create a directory for saving all output if it does not exist
output_dir = "simulation_outputs"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

###############################################
# Helper Functions
###############################################

def circular_mean(angles):
    """
    Compute the circular mean of a set of angles (in radians).
    """
    sin_sum = np.sum(np.sin(angles))
    cos_sum = np.sum(np.cos(angles))
    mean_angle = np.arctan2(sin_sum, cos_sum)
    return mean_angle

def mean_resultant_length(angles):
    """
    Compute the mean resultant length R of angles.
    R = 1/n * sqrt((sum cos(theta))^2 + (sum sin(theta))^2)
    """
    n = len(angles)
    C = np.sum(np.cos(angles))
    S = np.sum(np.sin(angles))
    R = np.sqrt((C**2 + S**2)) / n
    return R

def estimate_kappa(R):
    """
    Estimate the concentration parameter kappa for von Mises distribution
    given the resultant length R.
    Approximation from Fisher (1993).
    """
    if R < 0.53:
        kappa = 2 * R + R**3 + (5 * R**5) / 6
    elif R >= 0.53 and R < 0.85:
        kappa = -0.4 + 1.39*R + 0.43/(1-R)
    else:
        # Another approximation for larger R
        kappa = 1/(R**3 -4*R**2 +3*R)
        # This is a rough approximation; one can refine with more accurate formulas
        # or numerical methods. For demonstration, we use a simplified approach here.
        # In practice, use a more accurate formula or iterative solution.
    return kappa

def rayleigh_test(angles):
    """
    Conduct the Rayleigh test for uniformity of circular data.
    H0: Uniform distribution around the circle.
    Z = n*R^2, where R is resultant length.
    p-value = exp(-Z) * (1 + (2Z - Z^2)/(4n) - ...)
    For large n, a simpler approximation is often used.
    """
    n = len(angles)
    R = mean_resultant_length(angles)
    Z = (n * (R**2))
    # Approximate p-value for Rayleigh test
    # Using approximation: p = exp(-Z)*(1+(2Z-Z^2)/(4n) - ...)
    # For simplicity, we use the leading term:
    p_value = np.exp(-Z)
    return Z, p_value

def circular_reliability_function(angles, bins=50):
    """
    Estimate a circular reliability-like function by treating angles as "failure angles".
    We define a pseudo CDF F(theta) and then R(theta)=1-F(theta).
    """
    # Sort angles
    sorted_angles = np.sort(angles)
    n = len(angles)
    # Empirical CDF
    # Since angles are in [0, 2pi), we can create an empirical distribution
    # and treat them as "failure times" in a cycle.
    # R(theta) = Probability(component survives beyond angle theta
    # = 1 - F(theta)
    # We'll make a discrete approximation.
    theta_grid = np.linspace(0, 2*np.pi, bins)
    F = np.array([np.sum(sorted_angles <= th)/n for th in theta_grid])
    R = 1 - F
    return theta_grid, R

def hazard_function(angles, bins=50):
    """
    Estimate a hazard function for circular data:
    h(theta) = f(theta)/(1-F(theta))
    We'll approximate PDF f and CDF F from the angles using kernel density or simple hist.
    """
    sorted_angles = np.sort(angles)
    n = len(angles)
    theta_grid = np.linspace(0, 2*np.pi, bins)
    # Simple histogram-based PDF estimate
    counts, bin_edges = np.histogram(angles, bins=theta_grid, density=True)
    pdf = counts
    mid_points = 0.5*(bin_edges[1:] + bin_edges[:-1])
    cdf = np.cumsum(pdf * (mid_points[1]-mid_points[0])) # approximate cdf
    cdf = cdf / cdf[-1]  # normalize to 1

    # hazard = f(theta)/(1-F(theta))
    hazard = pdf / (1 - cdf)
    return mid_points, hazard

def circular_control_chart(angles, alpha=0.05):
    """
    Create a circular control chart assuming data ~ von Mises.
    1. Estimate mean direction and kappa.
    2. Determine control limits via quantiles of von Mises distribution.
    For simplicity, we will define control limits as mean +/- some quantile based on kappa.
    NOTE: von Mises quantiles would normally require numeric methods. Here, we approximate.
    """
    mu_est = circular_mean(angles)
    R = mean_resultant_length(angles)
    kappa_est = estimate_kappa(R)

    # For control limits, we want angles s.t. P(Theta < LCL)=alpha/2 and P(Theta > UCL)=alpha/2.
    # Since we know mu and kappa, we can invert the CDF of von Mises to find quantiles.
    # Here, for demonstration, we do a rough approximation:
    # von Mises distribution is approximately normal in the angular domain for large kappa.
    # Approx normal approximation:
    # If kappa is large, VM ~ Normal(mu, 1/kappa) on the circle.
    # Just for demonstration: LCL = mu - z * sqrt(1/kappa), UCL = mu + z * sqrt(1/kappa)
    # For a chosen alpha, z ~ normal quantile.
    # This is a rough approximation, a numeric approach would be better in practice.

    from scipy.stats import norm
    z = norm.ppf(1 - alpha/2)  # two-sided
    # Approximate standard deviation for circular data: sigma ~ 1/sqrt(kappa)
    sigma_approx = 1.0 / np.sqrt(kappa_est) if kappa_est > 0 else np.pi
    LCL = mu_est - z * sigma_approx
    UCL = mu_est + z * sigma_approx

    # Ensure angles are within [0, 2pi)
    LCL = LCL % (2*np.pi)
    UCL = UCL % (2*np.pi)

    return mu_est, LCL, UCL, kappa_est

def circular_linear_regression(angles, x):
    """
    Circular-linear regression:
    Model: theta = alpha + beta * x + error (wrapped on [0,2pi))
    We'll estimate alpha and beta by regressing cos(theta) and sin(theta) on x and solving.
    Steps:
    - Let Y1 = cos(theta), Y2 = sin(theta)
    - Model Y1 = a1 + b1 * x + error
             Y2 = a2 + b2 * x + error
    Fit two linear regressions and from a1,b1,a2,b2 we can reconstruct alpha,beta.
    This is a simplified approach, as full MLE would be more complicated.
    """
    X = np.column_stack((np.ones_like(x), x))
    # Fit linear models for cos(theta) and sin(theta)
    Y_cos = np.cos(angles)
    Y_sin = np.sin(angles)

    # OLS estimates: beta_hat = (X'X)^(-1) X'Y
    def ols_estimate(X, Y):
        XtX = X.T @ X
        XtY = X.T @ Y
        beta_hat = np.linalg.inv(XtX) @ XtY
        return beta_hat

    beta_cos = ols_estimate(X, Y_cos)
    beta_sin = ols_estimate(X, Y_sin)

    # From beta_cos = [a1, b1], beta_sin = [a2, b2]
    a1, b1 = beta_cos
    a2, b2 = beta_sin

    # Mean angle predicted for a given x:
    # mu_pred = atan2(a2 + b2*x, a1 + b1*x)
    # But we want to find an equivalent alpha and beta such that mu = alpha + beta*x.
    # This is tricky because alpha,beta are not simply from these regressions.
    # A rough approach: we can define alpha, beta by linearizing around mean x:
    x_mean = np.mean(x)
    # Evaluate direction at x_mean and x_mean+1:
    mu_xmean = np.arctan2(a2 + b2*x_mean, a1 + b1*x_mean)
    mu_xmean1 = np.arctan2(a2 + b2*(x_mean+1), a1 + b1*(x_mean+1))
    # Approx slope in angle w.r.t. x ~ mu_xmean1 - mu_xmean
    approx_beta = mu_xmean1 - mu_xmean
    approx_alpha = mu_xmean - approx_beta*x_mean

    # Residuals
    mu_fitted = np.arctan2(a2 + b2*x, a1 + b1*x)
    residuals = (angles - mu_fitted + np.pi) % (2*np.pi) - np.pi

    return (a1, b1, a2, b2, approx_alpha, approx_beta, mu_fitted, residuals)


###############################################
# Simulation Part
###############################################

# Set a random seed for reproducibility
np.random.seed(42)

# 1. Simulate failure times within a cycle (e.g., T=24 hours)
T_cycle = 24.0
n_failures = 300
# True parameters for failure angle distribution
mu_true_fail = np.pi       # mean direction = 180 degrees
kappa_true_fail = 3.0      # moderate concentration

# Generate angles from von Mises
failure_angles = vonmises.rvs(kappa_true_fail, loc=mu_true_fail, size=n_failures)
failure_times = (failure_angles / (2*np.pi)) * T_cycle

# 2. Circular statistics for failure data
mu_est_fail = circular_mean(failure_angles)
R_fail = mean_resultant_length(failure_angles)
kappa_est_fail = estimate_kappa(R_fail)
Z_fail, p_fail = rayleigh_test(failure_angles)

# 3. Reliability function
theta_grid_fail, Rfunc_fail = circular_reliability_function(failure_angles, bins=100)
theta_haz_fail, haz_fail = hazard_function(failure_angles, bins=100)

# Plot circular histogram of failure angles
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.hist(failure_angles, bins=20, density=True, alpha=0.7, color='blue')
ax.set_title("Circular Histogram of Simulated Failure Angles")
plt.savefig(os.path.join(output_dir, "circular_histogram_failure_angles.png"))
plt.close(fig)

# Plot Reliability function
fig, ax = plt.subplots()
ax.plot(theta_grid_fail, Rfunc_fail, 'r-', lw=2)
ax.set_title("Circular Reliability Function (Failure Data)")
ax.set_xlabel("Angle (radians)")
ax.set_ylabel("Reliability R(theta)")
plt.savefig(os.path.join(output_dir, "circular_reliability_failure.png"))
plt.close(fig)

# Plot Hazard function
fig, ax = plt.subplots()
ax.plot(theta_haz_fail, haz_fail, 'g-', lw=2)
ax.set_title("Estimated Hazard Function (Failure Data)")
ax.set_xlabel("Angle (radians)")
ax.set_ylabel("Hazard Rate")
plt.savefig(os.path.join(output_dir, "hazard_function_failure.png"))
plt.close(fig)

# Print summary table for failure data
failure_summary = pd.DataFrame({
    "Parameter": ["True Mean (rad)","Est Mean (rad)", "True kappa","Est kappa","Resultant Length","Rayleigh Z","Rayleigh p-value"],
    "Value": [mu_true_fail, mu_est_fail, kappa_true_fail, kappa_est_fail, R_fail, Z_fail, p_fail]
})
print("=== Failure Data Parameter Estimation Summary ===")
print(failure_summary)
print()

# 4. Simulate wind-like data (quality control scenario)
n_wind = 310
mu_true_wind = np.pi/2  # 90 degrees
kappa_true_wind = 4.0
wind_angles = vonmises.rvs(kappa_true_wind, loc=mu_true_wind, size=n_wind)

mu_est_wind = circular_mean(wind_angles)
R_wind = mean_resultant_length(wind_angles)
kappa_est_wind = estimate_kappa(R_wind)
Z_wind, p_wind = rayleigh_test(wind_angles)

theta_grid_wind, Rfunc_wind = circular_reliability_function(wind_angles, bins=100)
theta_haz_wind, haz_wind = hazard_function(wind_angles, bins=100)

# Plot polar histogram of wind angles
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.hist(wind_angles, bins=20, density=True, alpha=0.7, color='orange')
ax.set_title("Circular Histogram of Simulated Wind Angles")
plt.savefig(os.path.join(output_dir, "circular_histogram_wind_angles.png"))
plt.close(fig)

# Plot Reliability function for wind
fig, ax = plt.subplots()
ax.plot(theta_grid_wind, Rfunc_wind, 'r-', lw=2)
ax.set_title("Circular Reliability Function (Wind Data)")
ax.set_xlabel("Angle (radians)")
ax.set_ylabel("Reliability R(theta)")
plt.savefig(os.path.join(output_dir, "circular_reliability_wind.png"))
plt.close(fig)

# Plot Hazard function for wind
fig, ax = plt.subplots()
ax.plot(theta_haz_wind, haz_wind, 'g-', lw=2)
ax.set_title("Estimated Hazard Function (Wind Data)")
ax.set_xlabel("Angle (radians)")
ax.set_ylabel("Hazard Rate")
plt.savefig(os.path.join(output_dir, "hazard_function_wind.png"))
plt.close(fig)

# Print summary table for wind data
wind_summary = pd.DataFrame({
    "Parameter": ["True Mean (rad)","Est Mean (rad)", "True kappa","Est kappa","Resultant Length","Rayleigh Z","Rayleigh p-value"],
    "Value": [mu_true_wind, mu_est_wind, kappa_true_wind, kappa_est_wind, R_wind, Z_wind, p_wind]
})
print("=== Wind Data Parameter Estimation Summary ===")
print(wind_summary)
print()

# 5. Circular Control Chart for wind data
mu_ctrl, LCL_ctrl, UCL_ctrl, kappa_ctrl = circular_control_chart(wind_angles)

fig = plt.figure()
ax = fig.add_subplot(111, projection='polar')
ax.hist(wind_angles, bins=20, density=True, alpha=0.4, color='gray')
ax.axvline(mu_ctrl, color='red', linestyle='--', label='Mean Direction')
ax.axvline(LCL_ctrl, color='blue', linestyle='--', label='LCL')
ax.axvline(UCL_ctrl, color='blue', linestyle='--', label='UCL')
ax.set_title("Circular Control Chart (Wind Data)")
ax.legend()
plt.savefig(os.path.join(output_dir, "circular_control_chart_wind.png"))
plt.close(fig)

control_chart_summary = pd.DataFrame({
    "Parameter": ["Est Mean (rad)", "Est kappa", "LCL (rad)", "UCL (rad)"],
    "Value": [mu_ctrl, kappa_ctrl, LCL_ctrl, UCL_ctrl]
})
print("=== Circular Control Chart Parameters (Wind Data) ===")
print(control_chart_summary)
print()

# 6. Circular-linear regression
# Simulate a scenario: Defect angles depend on temperature linearly
n_defects = 200
temperatures = np.linspace(15, 30, n_defects)
# True relationship: mu(theta) = alpha + beta*Temp
alpha_true = np.pi/4  # 45 degrees
beta_true = np.pi/60  # slope
mu_defect = alpha_true + beta_true * temperatures
# Wrap angles to [0,2pi)
mu_defect = mu_defect % (2*np.pi)

# Add noise from von Mises with some kappa
kappa_reg = 5.0
defect_angles = vonmises.rvs(kappa_reg, loc=mu_defect, size=n_defects)

# Fit circular-linear regression
a1, b1, a2, b2, approx_alpha, approx_beta, mu_fitted, residuals = circular_linear_regression(defect_angles, temperatures)

# Scatter plot fitted vs observed
fig, ax = plt.subplots()
ax.scatter(mu_fitted, defect_angles, alpha=0.7, color='purple')
ax.plot([0,2*np.pi],[0,2*np.pi],'r--')
ax.set_xlim(0,2*np.pi)
ax.set_ylim(0,2*np.pi)
ax.set_title("Fitted vs Observed Defect Angles")
ax.set_xlabel("Fitted angles (rad)")
ax.set_ylabel("Observed angles (rad)")
plt.savefig(os.path.join(output_dir, "fitted_vs_observed_defect_angles.png"))
plt.close(fig)

# Residual plot
fig, ax = plt.subplots()
ax.scatter(mu_fitted, residuals, alpha=0.7, color='green')
ax.axhline(0, color='red', linestyle='--')
ax.set_title("Residuals vs Fitted Angles (Circular-Linear Regression)")
ax.set_xlabel("Fitted angles (rad)")
ax.set_ylabel("Residuals (rad)")
plt.savefig(os.path.join(output_dir, "residuals_vs_fitted_regression.png"))
plt.close(fig)

# Print regression summary
regression_summary = pd.DataFrame({
    "Parameter": ["True alpha", "True beta", "Approx alpha", "Approx beta",
                  "a1 (cos intercept)", "b1 (cos slope)", "a2 (sin intercept)", "b2 (sin slope)"],
    "Value": [alpha_true, beta_true, approx_alpha, approx_beta, a1, b1, a2, b2]
})
print("=== Circular-Linear Regression Summary ===")
print(regression_summary)
print()

# Printing some additional notes and confirmations
print("All figures have been saved to the '{}' directory.".format(output_dir))
print("The code has generated a comprehensive set of outputs, including multiple plots and tables.")


<ipython-input-1-bb1baa0e0618>:110: RuntimeWarning: divide by zero encountered in divide
  hazard = pdf / (1 - cdf)
<ipython-input-1-bb1baa0e0618>:110: RuntimeWarning: invalid value encountered in divide
  hazard = pdf / (1 - cdf)


=== Failure Data Parameter Estimation Summary ===
          Parameter         Value
0   True Mean (rad)  3.141593e+00
1    Est Mean (rad)  3.070852e+00
2        True kappa  3.000000e+00
3         Est kappa  3.101484e+00
4  Resultant Length  8.181270e-01
5        Rayleigh Z  2.007995e+02
6  Rayleigh p-value  6.221110e-88



<ipython-input-1-bb1baa0e0618>:110: RuntimeWarning: divide by zero encountered in divide
  hazard = pdf / (1 - cdf)
<ipython-input-1-bb1baa0e0618>:110: RuntimeWarning: invalid value encountered in divide
  hazard = pdf / (1 - cdf)


=== Wind Data Parameter Estimation Summary ===
          Parameter         Value
0   True Mean (rad)  1.570796e+00
1    Est Mean (rad)  1.623832e+00
2        True kappa  4.000000e+00
3         Est kappa  3.744129e+00
4  Resultant Length  8.542962e-01
5        Rayleigh Z  2.262448e+02
6  Rayleigh p-value  5.535117e-99

=== Circular Control Chart Parameters (Wind Data) ===
        Parameter     Value
0  Est Mean (rad)  1.623832
1       Est kappa  3.744129
2       LCL (rad)  0.610917
3       UCL (rad)  2.636746

=== Circular-Linear Regression Summary ===
            Parameter     Value
0          True alpha  0.785398
1           True beta  0.052360
2        Approx alpha  0.785952
3         Approx beta  0.051589
4  a1 (cos intercept)  0.603276
5      b1 (cos slope) -0.040926
6  a2 (sin intercept)  1.204181
7      b2 (sin slope) -0.017758

All figures have been saved to the 'simulation_outputs' directory.
The code has generated a comprehensive set of outputs, including multiple plots and ta

In [2]:
# Polar plot for Fitted vs. Observed Defect Angles
fig = plt.figure()
ax = fig.add_subplot(111, projection='polar')
ax.scatter(mu_fitted, defect_angles, alpha=0.7, color='purple', label="Data Points")
ax.set_theta_zero_location('N')  # Set 0 degrees to the top
ax.set_theta_direction(-1)  # Set clockwise direction for angles
ax.set_title("Fitted vs. Observed Defect Angles (Polar)")
plt.legend(loc="upper right")
plt.savefig(os.path.join(output_dir, "fitted_vs_observed_defect_angles_polar.png"))
plt.close(fig)


In [3]:
import matplotlib.pyplot as plt
import numpy as np

# Create a circular plot comparing fitted and observed angles
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': 'polar'})
ax.set_theta_zero_location('N')  # 0 degrees at the top
ax.set_theta_direction(-1)  # Clockwise angles
ax.scatter(mu_fitted, defect_angles, c='purple', alpha=0.7, label="Data Points")
ax.plot(np.linspace(0, 2*np.pi, 100), np.linspace(0, 2*np.pi, 100), 'r--', label="Perfect Fit Line")

ax.set_title("Fitted vs. Observed Defect Angles")
ax.legend(loc="upper right")
plt.savefig(os.path.join(output_dir, "fitted_vs_observed_defect_angles_circular.png"))
plt.close(fig)


The complete Python code for simulating **Daily Usage Cycles in Machinery** and analyzing the resulting data using circular statistics is as follows:

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import vonmises
from scipy.stats import norm

# Create a directory for saving all output
output_dir = "simulation_outputs_M"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

###############################################
# Helper Functions
###############################################

def circular_mean(angles):
    sin_sum = np.sum(np.sin(angles))
    cos_sum = np.sum(np.cos(angles))
    mean_angle = np.arctan2(sin_sum, cos_sum)
    return mean_angle

def mean_resultant_length(angles):
    n = len(angles)
    C = np.sum(np.cos(angles))
    S = np.sum(np.sin(angles))
    R = np.sqrt(C**2 + S**2) / n
    return R

def estimate_kappa(R):
    if R < 0.53:
        kappa = 2 * R + R**3 + (5 * R**5) / 6
    elif R < 0.85:
        kappa = -0.4 + 1.39 * R + 0.43 / (1 - R)
    else:
        kappa = 1 / (R**3 - 4 * R**2 + 3 * R)
    return kappa

def circular_reliability_function(angles, bins=50):
    sorted_angles = np.sort(angles)
    n = len(angles)
    theta_grid = np.linspace(0, 2 * np.pi, bins)
    F = np.array([np.sum(sorted_angles <= th) / n for th in theta_grid])
    R = 1 - F
    return theta_grid, R

def hazard_function(angles, bins=50):
    sorted_angles = np.sort(angles)
    theta_grid = np.linspace(0, 2 * np.pi, bins)
    counts, bin_edges = np.histogram(angles, bins=theta_grid, density=True)
    pdf = counts
    mid_points = 0.5 * (bin_edges[1:] + bin_edges[:-1])
    cdf = np.cumsum(pdf * (mid_points[1] - mid_points[0]))
    cdf = cdf / cdf[-1]
    hazard = pdf / (1 - cdf)
    return mid_points, hazard

def circular_control_chart(angles, alpha=0.05):
    mu_est = circular_mean(angles)
    R = mean_resultant_length(angles)
    kappa_est = estimate_kappa(R)
    z = norm.ppf(1 - alpha / 2)
    sigma_approx = 1.0 / np.sqrt(kappa_est) if kappa_est > 0 else np.pi
    LCL = (mu_est - z * sigma_approx) % (2 * np.pi)
    UCL = (mu_est + z * sigma_approx) % (2 * np.pi)
    return mu_est, LCL, UCL, kappa_est

###############################################
# Simulation
###############################################

# Simulate machinery failure times within a daily cycle
T_cycle = 24.0
n_failures = 300
mu_true_fail = np.pi  # Mean direction (180 degrees)
kappa_true_fail = 3.0  # Moderate concentration
failure_angles = vonmises.rvs(kappa_true_fail, loc=mu_true_fail, size=n_failures)
failure_times = (failure_angles / (2 * np.pi)) * T_cycle

# Analyze circular statistics
mu_est_fail = circular_mean(failure_angles)
R_fail = mean_resultant_length(failure_angles)
kappa_est_fail = estimate_kappa(R_fail)
theta_grid_fail, Rfunc_fail = circular_reliability_function(failure_angles)
theta_haz_fail, haz_fail = hazard_function(failure_angles)

# Plot Circular Histogram
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.hist(failure_angles, bins=20, density=True, alpha=0.7, color='blue')
ax.set_title("Circular Histogram of Simulated Machinery Failure Angles")
plt.savefig(os.path.join(output_dir, "circular_histogram_failures.png"))
plt.close(fig)

# Plot Reliability Function
fig, ax = plt.subplots()
ax.plot(theta_grid_fail, Rfunc_fail, 'r-', lw=2)
ax.set_title("Circular Reliability Function (Machinery Failures)")
ax.set_xlabel("Angle (radians)")
ax.set_ylabel("Reliability R(θ)")
plt.savefig(os.path.join(output_dir, "circular_reliability_function.png"))
plt.close(fig)

# Plot Hazard Function
fig, ax = plt.subplots()
ax.plot(theta_haz_fail, haz_fail, 'g-', lw=2)
ax.set_title("Hazard Function (Machinery Failures)")
ax.set_xlabel("Angle (radians)")
ax.set_ylabel("Hazard Rate")
plt.savefig(os.path.join(output_dir, "hazard_function.png"))
plt.close(fig)

# Create a Control Chart
mu_ctrl, LCL_ctrl, UCL_ctrl, kappa_ctrl = circular_control_chart(failure_angles)
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.hist(failure_angles, bins=20, density=True, alpha=0.4, color='gray')
ax.axvline(mu_ctrl, color='red', linestyle='--', label='Mean Direction')
ax.axvline(LCL_ctrl, color='blue', linestyle='--', label='LCL')
ax.axvline(UCL_ctrl, color='blue', linestyle='--', label='UCL')
ax.set_title("Circular Control Chart (Machinery Failures)")
ax.legend()
plt.savefig(os.path.join(output_dir, "circular_control_chart.png"))
plt.close(fig)

###############################################
# Circular-Linear Regression
###############################################

# Simulate relationship between temperature and defect angles
n_defects = 200
temperatures = np.linspace(15, 30, n_defects)
alpha_true = np.pi / 4  # Intercept (45 degrees)
beta_true = np.pi / 60  # Slope
mu_defect = (alpha_true + beta_true * temperatures) % (2 * np.pi)
kappa_reg = 5.0
defect_angles = vonmises.rvs(kappa_reg, loc=mu_defect, size=n_defects)

# Scatter Plot of Fitted vs. Observed Angles
fig, ax = plt.subplots()
ax.scatter(mu_defect, defect_angles, alpha=0.7, color='purple')
ax.plot([0, 2 * np.pi], [0, 2 * np.pi], 'r--', label="Perfect Agreement")
ax.set_xlim(0, 2 * np.pi)
ax.set_ylim(0, 2 * np.pi)
ax.set_title("Fitted vs Observed Defect Angles")
ax.set_xlabel("Fitted Angles (rad)")
ax.set_ylabel("Observed Angles (rad)")
ax.legend()
plt.savefig(os.path.join(output_dir, "fitted_vs_observed_angles.png"))
plt.close(fig)

print("Simulation complete. All results saved to:", output_dir)


<ipython-input-4-680d99d4bf00>:55: RuntimeWarning: divide by zero encountered in divide
  hazard = pdf / (1 - cdf)
<ipython-input-4-680d99d4bf00>:55: RuntimeWarning: invalid value encountered in divide
  hazard = pdf / (1 - cdf)


Simulation complete. All results saved to: simulation_outputs_M


In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import vonmises
from scipy.stats import norm

# Create a directory for saving all output
output_dir = "simulation_outputs_M"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

###############################################
# Helper Functions
###############################################

def circular_mean(angles):
    sin_sum = np.sum(np.sin(angles))
    cos_sum = np.sum(np.cos(angles))
    mean_angle = np.arctan2(sin_sum, cos_sum)
    return mean_angle

def mean_resultant_length(angles):
    n = len(angles)
    C = np.sum(np.cos(angles))
    S = np.sum(np.sin(angles))
    R = np.sqrt(C**2 + S**2) / n
    return R

def estimate_kappa(R):
    if R < 0.53:
        kappa = 2 * R + R**3 + (5 * R**5) / 6
    elif R < 0.85:
        kappa = -0.4 + 1.39 * R + 0.43 / (1 - R)
    else:
        kappa = 1 / (R**3 - 4 * R**2 + 3 * R)
    return kappa

def circular_reliability_function(angles, bins=50):
    sorted_angles = np.sort(angles)
    n = len(angles)
    theta_grid = np.linspace(0, 2 * np.pi, bins)
    F = np.array([np.sum(sorted_angles <= th) / n for th in theta_grid])
    R = 1 - F
    return theta_grid, R

def hazard_function(angles, bins=50):
    sorted_angles = np.sort(angles)
    theta_grid = np.linspace(0, 2 * np.pi, bins)
    counts, bin_edges = np.histogram(angles, bins=theta_grid, density=True)
    pdf = counts
    mid_points = 0.5 * (bin_edges[1:] + bin_edges[:-1])
    cdf = np.cumsum(pdf * (mid_points[1] - mid_points[0]))
    cdf = cdf / cdf[-1]
    hazard = pdf / (1 - cdf)
    return mid_points, hazard

def circular_control_chart(angles, alpha=0.05):
    mu_est = circular_mean(angles)
    R = mean_resultant_length(angles)
    kappa_est = estimate_kappa(R)
    z = norm.ppf(1 - alpha / 2)
    sigma_approx = 1.0 / np.sqrt(kappa_est) if kappa_est > 0 else np.pi
    LCL = (mu_est - z * sigma_approx) % (2 * np.pi)
    UCL = (mu_est + z * sigma_approx) % (2 * np.pi)
    return mu_est, LCL, UCL, kappa_est

def rayleigh_test(angles):
    n = len(angles)
    R = mean_resultant_length(angles)
    Z = n * (R**2)
    p_value = np.exp(-Z)
    return Z, p_value

###############################################
# Simulation
###############################################

# Simulate machinery failure times within a daily cycle
T_cycle = 24.0
n_failures = 300
mu_true_fail = np.pi  # Mean direction (180 degrees)
kappa_true_fail = 3.0  # Moderate concentration
failure_angles = vonmises.rvs(kappa_true_fail, loc=mu_true_fail, size=n_failures)
failure_times = (failure_angles / (2 * np.pi)) * T_cycle

# Analyze circular statistics
mu_est_fail = circular_mean(failure_angles)
R_fail = mean_resultant_length(failure_angles)
kappa_est_fail = estimate_kappa(R_fail)
theta_grid_fail, Rfunc_fail = circular_reliability_function(failure_angles)
theta_haz_fail, haz_fail = hazard_function(failure_angles)
Z_fail, p_fail = rayleigh_test(failure_angles)

# Plot Circular Histogram
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.hist(failure_angles, bins=20, density=True, alpha=0.7, color='blue')
ax.set_title("Circular Histogram of Simulated Machinery Failure Angles")
plt.savefig(os.path.join(output_dir, "circular_histogram_failures.png"))
plt.close(fig)

# Plot Reliability Function
fig, ax = plt.subplots()
ax.plot(theta_grid_fail, Rfunc_fail, 'r-', lw=2)
ax.set_title("Circular Reliability Function (Machinery Failures)")
ax.set_xlabel("Angle (radians)")
ax.set_ylabel("Reliability R(θ)")
plt.savefig(os.path.join(output_dir, "circular_reliability_function.png"))
plt.close(fig)

# Plot Hazard Function
fig, ax = plt.subplots()
ax.plot(theta_haz_fail, haz_fail, 'g-', lw=2)
ax.set_title("Hazard Function (Machinery Failures)")
ax.set_xlabel("Angle (radians)")
ax.set_ylabel("Hazard Rate")
plt.savefig(os.path.join(output_dir, "hazard_function.png"))
plt.close(fig)

# Create a Control Chart
mu_ctrl, LCL_ctrl, UCL_ctrl, kappa_ctrl = circular_control_chart(failure_angles)
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.hist(failure_angles, bins=20, density=True, alpha=0.4, color='gray')
ax.axvline(mu_ctrl, color='red', linestyle='--', label='Mean Direction')
ax.axvline(LCL_ctrl, color='blue', linestyle='--', label='LCL')
ax.axvline(UCL_ctrl, color='blue', linestyle='--', label='UCL')
ax.set_title("Circular Control Chart (Machinery Failures)")
ax.legend()
plt.savefig(os.path.join(output_dir, "circular_control_chart.png"))
plt.close(fig)

# Perform Circular-Linear Regression
n_defects = 200
temperatures = np.linspace(15, 30, n_defects)
alpha_true = np.pi / 4  # Intercept (45 degrees)
beta_true = np.pi / 60  # Slope
mu_defect = (alpha_true + beta_true * temperatures) % (2 * np.pi)
kappa_reg = 5.0
defect_angles = vonmises.rvs(kappa_reg, loc=mu_defect, size=n_defects)

# Scatter Plot of Fitted vs. Observed Angles
fig, ax = plt.subplots()
ax.scatter(mu_defect, defect_angles, alpha=0.7, color='purple')
ax.plot([0, 2 * np.pi], [0, 2 * np.pi], 'r--', label="Perfect Agreement")
ax.set_xlim(0, 2 * np.pi)
ax.set_ylim(0, 2 * np.pi)
ax.set_title("Fitted vs Observed Defect Angles")
ax.set_xlabel("Fitted Angles (rad)")
ax.set_ylabel("Observed Angles (rad)")
ax.legend()
plt.savefig(os.path.join(output_dir, "fitted_vs_observed_angles.png"))
plt.close(fig)

print("Simulation complete. All results saved to:", output_dir)


<ipython-input-5-6792a3a60113>:55: RuntimeWarning: divide by zero encountered in divide
  hazard = pdf / (1 - cdf)
<ipython-input-5-6792a3a60113>:55: RuntimeWarning: invalid value encountered in divide
  hazard = pdf / (1 - cdf)


Simulation complete. All results saved to: simulation_outputs_M


In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import vonmises
from scipy.stats import norm

# Create a directory for saving all output
output_dir = "simulation_outputs_M"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

###############################################
# Helper Functions
###############################################

def circular_mean(angles):
    sin_sum = np.sum(np.sin(angles))
    cos_sum = np.sum(np.cos(angles))
    mean_angle = np.arctan2(sin_sum, cos_sum)
    return mean_angle

def mean_resultant_length(angles):
    n = len(angles)
    C = np.sum(np.cos(angles))
    S = np.sum(np.sin(angles))
    R = np.sqrt(C**2 + S**2) / n
    return R

def estimate_kappa(R):
    if R < 0.53:
        kappa = 2 * R + R**3 + (5 * R**5) / 6
    elif R < 0.85:
        kappa = -0.4 + 1.39 * R + 0.43 / (1 - R)
    else:
        kappa = 1 / (R**3 - 4 * R**2 + 3 * R)
    return kappa

def circular_reliability_function(angles, bins=50):
    sorted_angles = np.sort(angles)
    n = len(angles)
    theta_grid = np.linspace(0, 2 * np.pi, bins)
    F = np.array([np.sum(sorted_angles <= th) / n for th in theta_grid])
    R = 1 - F
    return theta_grid, R

def hazard_function(angles, bins=50):
    sorted_angles = np.sort(angles)
    theta_grid = np.linspace(0, 2 * np.pi, bins)
    counts, bin_edges = np.histogram(angles, bins=theta_grid, density=True)
    pdf = counts
    mid_points = 0.5 * (bin_edges[1:] + bin_edges[:-1])
    cdf = np.cumsum(pdf * (mid_points[1] - mid_points[0]))
    cdf = cdf / cdf[-1]
    hazard = pdf / (1 - cdf)
    return mid_points, hazard

def circular_control_chart(angles, alpha=0.05):
    mu_est = circular_mean(angles)
    R = mean_resultant_length(angles)
    kappa_est = estimate_kappa(R)
    z = norm.ppf(1 - alpha / 2)
    sigma_approx = 1.0 / np.sqrt(kappa_est) if kappa_est > 0 else np.pi
    LCL = (mu_est - z * sigma_approx) % (2 * np.pi)
    UCL = (mu_est + z * sigma_approx) % (2 * np.pi)
    return mu_est, LCL, UCL, kappa_est

def rayleigh_test(angles):
    n = len(angles)
    R = mean_resultant_length(angles)
    Z = n * (R**2)
    p_value = np.exp(-Z)
    return Z, p_value

###############################################
# Simulation
###############################################

# Simulate machinery failure times within a daily cycle
T_cycle = 24.0
n_failures = 300
mu_true_fail = np.pi  # Mean direction (180 degrees)
kappa_true_fail = 3.0  # Moderate concentration
failure_angles = vonmises.rvs(kappa_true_fail, loc=mu_true_fail, size=n_failures)
failure_times = (failure_angles / (2 * np.pi)) * T_cycle

# Analyze circular statistics
mu_est_fail = circular_mean(failure_angles)
R_fail = mean_resultant_length(failure_angles)
kappa_est_fail = estimate_kappa(R_fail)
theta_grid_fail, Rfunc_fail = circular_reliability_function(failure_angles)
theta_haz_fail, haz_fail = hazard_function(failure_angles)
Z_fail, p_fail = rayleigh_test(failure_angles)

# Plot Circular Histogram
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.hist(failure_angles, bins=20, density=True, alpha=0.7, color='blue')
ax.set_title("Circular Histogram of Simulated Machinery Failure Angles")
plt.savefig(os.path.join(output_dir, "circular_histogram_failures.png"))
plt.close(fig)

# Plot Reliability Function
fig, ax = plt.subplots()
ax.plot(theta_grid_fail, Rfunc_fail, 'r-', lw=2)
ax.set_title("Circular Reliability Function (Machinery Failures)")
ax.set_xlabel("Angle (radians)")
ax.set_ylabel("Reliability R(θ)")
plt.savefig(os.path.join(output_dir, "circular_reliability_function.png"))
plt.close(fig)

# Plot Hazard Function
fig, ax = plt.subplots()
ax.plot(theta_haz_fail, haz_fail, 'g-', lw=2)
ax.set_title("Hazard Function (Machinery Failures)")
ax.set_xlabel("Angle (radians)")
ax.set_ylabel("Hazard Rate")
plt.savefig(os.path.join(output_dir, "hazard_function.png"))
plt.close(fig)

# Create a Control Chart
mu_ctrl, LCL_ctrl, UCL_ctrl, kappa_ctrl = circular_control_chart(failure_angles)
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.hist(failure_angles, bins=20, density=True, alpha=0.4, color='gray')
ax.axvline(mu_ctrl, color='red', linestyle='--', label='Mean Direction')
ax.axvline(LCL_ctrl, color='blue', linestyle='--', label='LCL')
ax.axvline(UCL_ctrl, color='blue', linestyle='--', label='UCL')
ax.set_title("Circular Control Chart (Machinery Failures)")
ax.legend()
plt.savefig(os.path.join(output_dir, "circular_control_chart.png"))
plt.close(fig)

# Perform Circular-Linear Regression
n_defects = 200
temperatures = np.linspace(15, 30, n_defects)
alpha_true = np.pi / 4  # Intercept (45 degrees)
beta_true = np.pi / 60  # Slope
mu_defect = (alpha_true + beta_true * temperatures) % (2 * np.pi)
kappa_reg = 5.0
defect_angles = vonmises.rvs(kappa_reg, loc=mu_defect, size=n_defects)

# Scatter Plot of Fitted vs. Observed Angles
fig, ax = plt.subplots()
ax.scatter(mu_defect, defect_angles, alpha=0.7, color='purple')
ax.plot([0, 2 * np.pi], [0, 2 * np.pi], 'r--', label="Perfect Agreement")
ax.set_xlim(0, 2 * np.pi)
ax.set_ylim(0, 2 * np.pi)
ax.set_title("Fitted vs Observed Defect Angles")
ax.set_xlabel("Fitted Angles (rad)")
ax.set_ylabel("Observed Angles (rad)")
ax.legend()
plt.savefig(os.path.join(output_dir, "fitted_vs_observed_angles.png"))
plt.close(fig)

print("Simulation complete. All results saved to:", output_dir)


<ipython-input-6-6792a3a60113>:55: RuntimeWarning: divide by zero encountered in divide
  hazard = pdf / (1 - cdf)
<ipython-input-6-6792a3a60113>:55: RuntimeWarning: invalid value encountered in divide
  hazard = pdf / (1 - cdf)


Simulation complete. All results saved to: simulation_outputs_M


In [7]:
import pandas as pd
import os

# Ensure the output directory exists
output_dir = "simulation_outputs_M"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Create and save the Failure Data Parameter Estimation Table
failure_summary = pd.DataFrame({
    "Parameter": ["True Mean (rad)", "Est Mean (rad)", "True kappa", "Est kappa",
                  "Resultant Length", "Rayleigh Z", "Rayleigh p-value"],
    "Value": [mu_true_fail, mu_est_fail, kappa_true_fail, kappa_est_fail, R_fail, Z_fail, p_fail]
})
failure_summary.to_csv(os.path.join(output_dir, "failure_data_summary.csv"), index=False)

# Create and save the Circular Control Chart Table
control_chart_summary = pd.DataFrame({
    "Parameter": ["Est Mean (rad)", "Est kappa", "LCL (rad)", "UCL (rad)"],
    "Value": [mu_ctrl, kappa_ctrl, LCL_ctrl, UCL_ctrl]
})
control_chart_summary.to_csv(os.path.join(output_dir, "control_chart_summary.csv"), index=False)

# Create and save the Circular-Linear Regression Table
regression_summary = pd.DataFrame({
    "Parameter": ["True alpha", "True beta", "Estimated Mean Angle"],
    "Value": [alpha_true, beta_true, circular_mean(mu_defect)]
})
regression_summary.to_csv(os.path.join(output_dir, "regression_summary.csv"), index=False)

print("Tables saved successfully as CSV files in the '{}' directory.".format(output_dir))


Tables saved successfully as CSV files in the 'simulation_outputs_M' directory.


Real Data

In [ ]:
pip install pycircstat matplotlib


In [8]:
!pip install nose

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 3.8 MB/s eta 0:00:00


To represent the Fitted vs. Observed Defect Angles in a circular-circular coordinate system, We can modify the Python code as follows. This will create a polar plot where both axes are circular, showing how the fitted and observed angles correspond.

In [9]:
# Plot Fitted vs Observed Defect Angles in Circular-Circular Coordinates
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(8, 8))

# Scatter plot for fitted vs observed angles
ax.scatter(mu_fitted, defect_angles, alpha=0.7, color='purple', label='Data Points')

# Add a dashed line to represent perfect agreement
angles = np.linspace(0, 2 * np.pi, 100)
ax.plot(angles, angles, 'r--', label="Perfect Agreement")

# Set the title and labels
ax.set_title("Fitted vs. Observed Defect Angles (Circular-Circular Coordinates)")
ax.legend(loc="upper left", bbox_to_anchor=(1.1, 1.05))
plt.savefig(os.path.join(output_dir, "fitted_vs_observed_circular_circular.png"))
plt.close(fig)


In [10]:
# Plot Fitted vs Observed Defect Angles in Circular-Circular Coordinates with Residuals
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(8, 8))

# Scatter plot for fitted vs observed angles
ax.scatter(mu_fitted, defect_angles, alpha=0.7, color='purple', label='Data Points')

# Add a dashed line to represent perfect agreement
angles = np.linspace(0, 2 * np.pi, 100)
ax.plot(angles, angles, 'r--', label="Perfect Agreement")

# Add residual arrows
for fitted, observed in zip(mu_fitted, defect_angles):
    ax.plot([fitted, fitted], [fitted, observed], 'gray', alpha=0.5)  # Residual lines
    ax.scatter([fitted], [observed], color='orange', alpha=0.7)  # Residual endpoints

# Set the title and labels
ax.set_title("Fitted vs. Observed Defect Angles (Circular-Circular with Residuals)")
ax.legend(loc="upper left", bbox_to_anchor=(1.1, 1.05))
plt.savefig(os.path.join(output_dir, "fitted_vs_observed_circular_residuals.png"))
plt.close(fig)


Residuals circular Distribution

In [11]:
# Circular plot for residuals
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(8, 8))

# Compute residuals (wrapped within [-pi, pi])
circular_residuals = (defect_angles - mu_fitted + np.pi) % (2 * np.pi) - np.pi

# Plot histogram of residuals
ax.hist(circular_residuals, bins=20, density=True, alpha=0.7, color='green', label='Residuals')
ax.axvline(0, color='red', linestyle='--', label='Zero Residual Line')

# Set the title and legend
ax.set_title("Circular Residuals Distribution")
ax.legend(loc="upper left", bbox_to_anchor=(1.1, 1.05))
plt.savefig(os.path.join(output_dir, "circular_residuals_distribution.png"))
plt.close(fig)


Python Code for Residual Distribution with von Mises Overlay

In [12]:
from scipy.stats import vonmises

# Circular plot for residuals with von Mises overlay
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(8, 8))

# Compute residuals (wrapped within [-pi, pi])
circular_residuals = (defect_angles - mu_fitted + np.pi) % (2 * np.pi) - np.pi

# Plot histogram of residuals
counts, bin_edges, _ = ax.hist(circular_residuals, bins=20, density=True, alpha=0.7, color='green', label='Residuals')

# Fit a von Mises distribution to the residuals
kappa_fit, loc_fit = estimate_kappa(mean_resultant_length(circular_residuals)), circular_mean(circular_residuals)

# Overlay von Mises density
theta = np.linspace(-np.pi, np.pi, 500)
von_mises_density = vonmises.pdf(theta, kappa_fit, loc=loc_fit)
ax.plot(theta, von_mises_density, 'r-', label=f'Von Mises Fit (kappa={kappa_fit:.2f}, loc={loc_fit:.2f})')

# Add reference line for zero residuals
ax.axvline(0, color='blue', linestyle='--', label='Zero Residual Line')

# Set title and legend
ax.set_title("Circular Residuals Distribution with Von Mises Overlay")
ax.legend(loc="upper left", bbox_to_anchor=(1.1, 1.05))

# Save the plot
plt.savefig(os.path.join(output_dir, "circular_residuals_with_vonmises.png"))
plt.close(fig)
